# ТВиМС задание 1

## Вариант 14

Интервальные оценки

***

## Setup:

In [1]:
import math
import matplotlib.pyplot as plt

import base
import vs_analysis
import taskdata

from ipywidgets import interact, interact_manual, fixed

In [2]:
# change this to set the best plot appearance on your display
plt.rcParams['figure.figsize'] = [18, 10]

## Точечные оценки:

### Матожидание и дисперсия выборки в зависимости от ее размера:

In [3]:
def get_mu_and_s_squared_for_varseries(n):
    vs = base.generate_varseries(taskdata.get_y, n)
    mu = vs_analysis.expected_value(vs)
    s_sq = vs_analysis.variance(vs)
    return mu, s_sq

In [4]:
interact(get_mu_and_s_squared_for_varseries, n=(2, 1_000_000))

interactive(children=(IntSlider(value=500001, description='n', max=1000000, min=2), Output()), _dom_classes=('…

<function __main__.get_mu_and_s_squared_for_varseries(n)>

### Аналитические расчеты матожидания и дисперсии:

![title](imgs/img3.jpg)

In [5]:
mu = 2*math.log(2) / math.pi
print(mu)

0.4412712003053032


In [6]:
var = 4/math.pi - 1 - 4/math.pi**2 * math.log(2)**2
print(var)

0.07851927251627977


### Зависимость величины отклонений от размера выборки:

In [7]:
def deviations(n):
    vs_mu, vs_s_sq = get_mu_and_s_squared_for_varseries(n)
    return abs(mu - vs_mu), abs(var - vs_s_sq)

interact(deviations, n=(2, 1_000_000))

interactive(children=(IntSlider(value=500001, description='n', max=1000000, min=2), Output()), _dom_classes=('…

<function __main__.deviations(n)>

## Интервальные оценки:

In [8]:
def plot_mu_stuff(n, var_known=True):
    vs = base.generate_varseries(taskdata.get_y, n)
    
    confidences = [0.01*i for i in range(101)]

    if var_known:
        intervals = [vs_analysis.confidence_interval(vs, confidence, 'матожидание', var=var) for confidence in confidences]
    else:
        intervals = [vs_analysis.confidence_interval(vs, confidence, 'матожидание') for confidence in confidences]

    intervals_lower_bounds = [interval[0] for interval in intervals]
    intervals_upper_bounds = [interval[1] for interval in intervals]

    # fig, ax = plt.subplots(figsize=(12, 8))
    plt.ylim(taskdata.y_a, taskdata.y_b)
    plt.plot([0, 1], [mu, mu], 'k--', label='аналитическое матожидание')
    plt.scatter(confidences, intervals_lower_bounds, label='нижние границы интервалов')
    plt.scatter(confidences, intervals_upper_bounds, label='верхние границы интервалов')
    plt.legend()
    plt.show()


def plot_var_stuff(n, mu_known=True):
    vs = base.generate_varseries(taskdata.get_y, n)

    confidences = [0.01*i for i in range(101)]

    if mu_known:
        intervals = [vs_analysis.confidence_interval(vs, confidence, 'дисперсия', mu=mu) for confidence in confidences]
    else:
        intervals = [vs_analysis.confidence_interval(vs, confidence, 'дисперсия') for confidence in confidences]

    intervals_lower_bounds = [interval[0] for interval in intervals]
    intervals_upper_bounds = [interval[1] for interval in intervals]

    plt.ylim(0, 3*var)
    plt.plot([0, 1], [var, var], 'k--', label='аналитическая дисперсия')
    plt.scatter(confidences, intervals_lower_bounds, label='нижние границы интервалов')
    plt.scatter(confidences, intervals_upper_bounds, label='верхние границы интервалов')
    plt.legend()
    plt.show()


### Доверительные интервалы (матожидание):

In [9]:
interact_manual(plot_mu_stuff, n=(5, 1000), var_known=[True, False])

interactive(children=(IntSlider(value=502, description='n', max=1000, min=5), Dropdown(description='var_known'…

<function __main__.plot_mu_stuff(n, var_known=True)>

### Доверительные интервалы (дисперсия):

In [10]:
interact_manual(plot_var_stuff, n=(5, 1000), mu_known=[True, False])

interactive(children=(IntSlider(value=502, description='n', max=1000, min=5), Dropdown(description='mu_known',…

<function __main__.plot_var_stuff(n, mu_known=True)>

### Завсисимость величины доверительных интервалов от объема выборки при фиксированной доверительной вероятности:

In [11]:
def intlen_mu(n, confidence, var_known=True):
    vs = base.generate_varseries(taskdata.get_y, n)
    interval = vs_analysis.confidence_interval(vs, confidence, 'матожидание', var=(var if var_known else None))
    return interval[1] - interval[0]


def intlen_var(n, confidence, mu_known=True):
    vs = base.generate_varseries(taskdata.get_y, n)
    interval = vs_analysis.confidence_interval(vs, confidence, 'дисперсия', mu=(mu if mu_known else None))
    return interval[1] - interval[0]


SAMPLE_NUM = 64


def plot_more_mu_stuff(n_min, n_max, n_step, confidence, var_known=True):
    ns = [n for n in range(n_min, n_max+1, n_step)]
    intlens = [0]*len(ns)
    for (i, n) in enumerate(ns):
        for _ in range(SAMPLE_NUM):
            intlens[i] += intlen_mu(n, confidence, var_known=var_known)
        intlens[i] /= SAMPLE_NUM

    plt.scatter(ns, intlens)
    plt.show()


def plot_more_var_stuff(n_min, n_max, n_step, confidence, mu_known=True):
    ns = [n for n in range(n_min, n_max+1, n_step)]
    intlens = [0]*len(ns)
    for (i, n) in enumerate(ns):
        for _ in range(SAMPLE_NUM):
            intlens[i] += intlen_var(n, confidence, mu_known=mu_known)
        intlens[i] /= SAMPLE_NUM

    plt.scatter(ns, intlens)
    plt.show()

In [12]:
interact_manual(plot_more_mu_stuff, n_min=fixed(5), n_max=fixed(105), n_step=fixed(1), confidence=(0.01, 0.99, 0.01), var_known=[True, False])

interactive(children=(FloatSlider(value=0.5, description='confidence', max=0.99, min=0.01, step=0.01), Dropdow…

<function __main__.plot_more_mu_stuff(n_min, n_max, n_step, confidence, var_known=True)>

In [13]:
interact_manual(plot_more_var_stuff, n_min=fixed(5), n_max=fixed(105), n_step=fixed(1), confidence=(0.01, 0.99, 0.01), mu_known=[True, False])

interactive(children=(FloatSlider(value=0.5, description='confidence', max=0.99, min=0.01, step=0.01), Dropdow…

<function __main__.plot_more_var_stuff(n_min, n_max, n_step, confidence, mu_known=True)>